In [3]:
'''
Analyzing NF-CSE-CIC-IDS2018 dataset, 
fixing the L7)PROTOCOL column to be numerical,
and binning top categorical values for each column so DL model can be trained on it.
'''
#imoprt pandas and numpy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

df = pd.read_parquet('NF-CSE-CIC-IDS2018-v2.parquet')
df.drop(['L4_SRC_PORT', 'DNS_QUERY_ID','FTP_COMMAND_RET_CODE','IPV4_DST_ADDR', 'IPV4_SRC_ADDR'], axis=1, inplace=True)
display(df.shape)


(18893708, 40)

In [4]:
# check for null values in every column
df.isnull().sum()

L4_DST_PORT                    0
PROTOCOL                       0
L7_PROTO                       0
IN_BYTES                       0
IN_PKTS                        0
OUT_BYTES                      0
OUT_PKTS                       0
TCP_FLAGS                      0
CLIENT_TCP_FLAGS               0
SERVER_TCP_FLAGS               0
FLOW_DURATION_MILLISECONDS     0
DURATION_IN                    0
DURATION_OUT                   0
MIN_TTL                        0
MAX_TTL                        0
LONGEST_FLOW_PKT               0
SHORTEST_FLOW_PKT              0
MIN_IP_PKT_LEN                 0
MAX_IP_PKT_LEN                 0
SRC_TO_DST_SECOND_BYTES        0
DST_TO_SRC_SECOND_BYTES        0
RETRANSMITTED_IN_BYTES         0
RETRANSMITTED_IN_PKTS          0
RETRANSMITTED_OUT_BYTES        0
RETRANSMITTED_OUT_PKTS         0
SRC_TO_DST_AVG_THROUGHPUT      0
DST_TO_SRC_AVG_THROUGHPUT      0
NUM_PKTS_UP_TO_128_BYTES       0
NUM_PKTS_128_TO_256_BYTES      0
NUM_PKTS_256_TO_512_BYTES      0
NUM_PKTS_5

In [5]:
# bin the top 100 ports in L4_DST_PORT a
top_100_ports = df['L4_DST_PORT'].value_counts().index[:100]
df.loc[~df['L4_DST_PORT'].isin(top_100_ports), 'L4_DST_PORT'] = 'other'
# one hot encode the column
df = pd.concat([df, pd.get_dummies(df['L4_DST_PORT'], prefix='L4_DST_PORT')], axis=1)
df.drop(['L4_DST_PORT'], axis=1, inplace=True)



/tmp/ipykernel_2747821/252107252.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[~df['L4_DST_PORT'].isin(top_100_ports), 'L4_DST_PORT'] = 'other'


In [6]:
#show if L7_PROTO has floating values
df['L7_PROTO'].value_counts()


L7_PROTO
0.000      11051460
7.000       2561575
5.126       1345675
41.000       519693
91.178       440160
             ...   
78.178            1
92.700            1
91.199            1
152.000           1
171.000           1
Name: count, Length: 287, dtype: int64

In [7]:
# convert L7_PROTO to int
df['L7_PROTO'] = df['L7_PROTO'].astype(int)

In [9]:
# bin the top 100 ports in L7_PROTO
top_100_protos = df['L7_PROTO'].value_counts().index[:100]
df.loc[~df['L7_PROTO'].isin(top_100_protos), 'L7_PROTO'] = 'other'


/tmp/ipykernel_2747821/1513551385.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[~df['L7_PROTO'].isin(top_100_protos), 'L7_PROTO'] = 'other'


In [ ]:
# bin the top 100 values in L7_PROTO and add thes
top_100_protos = df['L7_PROTO'].value_counts().index[:100]
df.loc[~df['L7_PROTO'].isin(top_100_protos), 'L7_PROTO'] = 'other'
# one hot encode the column
df = pd.concat([df, pd.get_dummies(df['L7_PROTO'], prefix='L7_PROTO')], axis=1)
df.drop(['L7_PROTO'], axis=1, inplace=True)

In [11]:
df['Attack'].value_counts()

Attack
Benign                      16635567
DDOS attack-HOIC             1080858
DoS attacks-Hulk              432648
DDoS attacks-LOIC-HTTP        307300
Bot                           143097
Infilteration                 116361
SSH-Bruteforce                 94979
DoS attacks-GoldenEye          27723
FTP-BruteForce                 25933
DoS attacks-SlowHTTPTest       14116
DoS attacks-Slowloris           9512
Brute Force -Web                2143
DDOS attack-LOIC-UDP            2112
Brute Force -XSS                 927
SQL Injection                    432
Name: count, dtype: int64

In [12]:
# get all values in the Attack column that are 'Brute Force -XSS', only show not duplicated values
xss_df = df[df['Attack'] == 'Brute Force -XSS'].drop_duplicates()


In [13]:
# analyzing misclassified attacks labeled as 'Brute Force -XSS', out_bytes contains zeros
xss_df.head()

,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_TYPE,DNS_TTL_ANSWER,Label,Attack
93187,80,6,7,503,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
102585,80,6,7,479,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
106249,80,6,7,511,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
114444,80,6,7,554,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
126337,80,6,7,501,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS


In [14]:
xss_df

,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_TYPE,DNS_TTL_ANSWER,Label,Attack
93187,80,6,7,503,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
102585,80,6,7,479,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
106249,80,6,7,511,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
114444,80,6,7,554,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
126337,80,6,7,501,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18737079,80,6,7,527,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
18768468,80,6,7,503,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
18781866,80,6,7,535,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
18813607,80,6,7,500,5,1147,5,219,219,27,...,1,0,65535,26883,0,0,0,0,1,Brute Force -XSS
